In [1]:
import requests as re
import pandas as pd
import time

## Functions

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.81 Safari/537.36 Edg/104.0.1293.47",
    "Accept-Language": "en-CA,en-US;q=0.9,en;q=0.8",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": "RGAPI-18ecbb33-6603-4c47-8347-5a70cff963b3"
}

In [11]:
def summoner_puuids(summoner_names_list, server):
    puuid_list = []
    for lst in summoner_names_list:
        temp_lst = []
        for summoner_name in lst:
            try:
                url = 'https://' + server + '.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + summoner_name
                summoner_info = re.get(url, headers=headers).json()

                puuid = summoner_info['puuid']
                temp_lst.append(puuid)
            except:
                pass
        
        puuid_list.append(temp_lst)

    time.sleep(120)
    return puuid_list

In [24]:
def match_ids(puuid_lists, region):
    # Create empty list for all matchIDs in European Dataset
    match_id_list = []

    for lst in puuid_lists:
    # Send a request to get the matchIDs for the latest 30 games played by each player
        temp_lst = []
        for puuid in lst:
            url = 'https://' + region + '.api.riotgames.com/lol/match/v5/matches/by-puuid/' + puuid + '/ids?start=0&count=20'
            matchIDs = re.get(url, headers=headers).json()
            # Access each matchID in the 20 latest games played
            for matchID in matchIDs:
                # Append the matchID to the master list EUW_matchID
                temp_lst.append(matchID)
        match_id_list.append(temp_lst)

    time.sleep(120)
    return(match_id_list)

In [27]:
def match_data(matchid_lists, region):

    matches_df = pd.DataFrame(columns=['MATCH_ID', 
       'BSUM1_ID', 'BTOP_CHAMP_ID', 'BSUM2_ID','BJGL_CHAMP_ID', 'BSUM3_ID', 'BMID_CHAMP_ID', 
       'BSUM4_ID', 'BBOT_CHAMP_ID', 'BSUM5_ID', 'BSUP_CHAMP_ID', 
       'BBAN1', 'BBAN2', 'BBAN3', 'BBAN4','BBAN5', 
       'RSUM1_ID', 'RTOP_CHAMP_ID', 'RSUM2_ID', 'RJGL_CHAMP_ID', 'RSUM3_ID', 'RMID_CHAMP_ID', 
       'RSUM4_ID', 'RBOT_CHAMP_ID', 'RSUM5_ID', 'RSUP_CHAMP_ID', 
       'RBAN1', 'RBAN2', 'RBAN3', 'RBAN4', 'RBAN5', 'FIRST_BLOOD','BCHAMP_KILLS' , 'RCHAMP_KILLS',
       'FIRST_DRAG', 'BDRAG_KILLS' , 'RDRAG_KILLS', 
       'FIRST_HERALD', 'BHERALD_KILLS', 'RHERALD_KILLS', 
       'FIRST_TOWER' ,'BTOWER_KILLS' , 'RTOWER_KILLS',
       'FIRST_BARON','BBARON_KILLS' , 'RBARON_KILLS',
       'FIRST_INHIB','BINHIB_KILLS' , 'RINHIB_KILLS', 
       'WINNER'])

    for lst in matchid_lists:

        # Send a request to the Riot Games API to get the match info for each matchID
        for matchID in lst:
            try:
                url = 'https://'+ region + '.api.riotgames.com/lol/match/v5/matches/' + matchID
                match_info = re.get(url, headers=headers).json()

                # Create a temporary DataFrame
                match = pd.DataFrame()

                # Get the MatchID
                match['MATCH_ID'] = [match_info['metadata']['matchId']]

                # Get the Summoner ID and ChampionID for each member on the Blue Team
                match['BSUM1_ID'] = [match_info['info']['participants'][0]['summonerId']]
                match['BTOP_CHAMP_ID'] = [match_info['info']['participants'][0]['championId']]

                match['BSUM2_ID'] = match_info['info']['participants'][1]['summonerId']
                match['BJGL_CHAMP_ID'] = match_info['info']['participants'][1]['championId']

                match['BSUM3_ID'] = match_info['info']['participants'][2]['summonerId']
                match['BMID_CHAMP_ID'] = match_info['info']['participants'][2]['championId']

                match['BSUM4_ID'] = match_info['info']['participants'][3]['summonerId']
                match['BBOT_CHAMP_ID'] = match_info['info']['participants'][3]['championId']

                match['BSUM5_ID'] = match_info['info']['participants'][4]['summonerId']
                match['BSUP_CHAMP_ID'] = match_info['info']['participants'][4]['championId']

                # Get the Champion ID for the 5 blue team bans
                match['BBAN1'] = match_info['info']['teams'][0]['bans'][0]['championId']
                match['BBAN2'] = match_info['info']['teams'][0]['bans'][1]['championId']
                match['BBAN3'] = match_info['info']['teams'][0]['bans'][2]['championId']
                match['BBAN4'] = match_info['info']['teams'][0]['bans'][3]['championId']
                match['BBAN5'] = match_info['info']['teams'][0]['bans'][4]['championId']

                # Get the Summoner ID and ChampionID for each member on the Red Team
                match['RSUM1_ID'] = match_info['info']['participants'][5]['summonerId']
                match['RTOP_CHAMP_ID'] = match_info['info']['participants'][5]['championId']

                match['RSUM2_ID'] = match_info['info']['participants'][6]['summonerId']
                match['RJGL_CHAMP_ID'] = match_info['info']['participants'][6]['championId']

                match['RSUM3_ID'] = match_info['info']['participants'][7]['summonerId']
                match['RMID_CHAMP_ID'] = match_info['info']['participants'][7]['championId']

                match['RSUM4_ID'] = match_info['info']['participants'][8]['summonerId']
                match['RBOT_CHAMP_ID'] = match_info['info']['participants'][8]['championId']

                match['RSUM5_ID'] = match_info['info']['participants'][9]['summonerId']
                match['RSUP_CHAMP_ID'] = match_info['info']['participants'][9]['championId']

                # Get the Champion ID for the 5 red team bans
                match['RBAN1'] = match_info['info']['teams'][1]['bans'][0]['championId']
                match['RBAN2'] = match_info['info']['teams'][1]['bans'][1]['championId']
                match['RBAN3'] = match_info['info']['teams'][1]['bans'][2]['championId']
                match['RBAN4'] = match_info['info']['teams'][1]['bans'][3]['championId']
                match['RBAN5'] = match_info['info']['teams'][1]['bans'][4]['championId']

                # Assign who got First Blood and get the total kills for each team
                if match_info['info']['teams'][0]['objectives']['champion']['first'] == True:
                    match['FIRST_BLOOD'] = ['BLUE']
                elif match_info['info']['teams'][1]['objectives']['champion']['first'] == True:
                    match['FIRST_BLOOD'] = ['RED']
                else:
                    match['FIRST_BLOOD'] = ['NONE']
    
                match['BCHAMP_KILLS'] = match_info['info']['teams'][0]['objectives']['champion']['kills']
                match['RCHAMP_KILLS'] = match_info['info']['teams'][1]['objectives']['champion']['kills']

                # Assign who got First Dragon and get the total dragon kills for each team
                if match_info['info']['teams'][0]['objectives']['dragon']['first'] == True:
                    match['FIRST_DRAG'] = ['BLUE']
                elif match_info['info']['teams'][1]['objectives']['dragon']['first'] == True:
                    match['FIRST_DRAG'] = ['RED']
                else:
                    match['FIRST_DRAG'] = ['NONE']

                match['BDRAG_KILLS'] = match_info['info']['teams'][0]['objectives']['dragon']['kills']
                match['RDRAG_KILLS'] = match_info['info']['teams'][1]['objectives']['dragon']['kills']

                # Assign who got First Rift Herald and get the total rift herald kills for each team
                if match_info['info']['teams'][0]['objectives']['riftHerald']['first'] == True:
                    match['FIRST_HERALD'] = ['BLUE']
                elif match_info['info']['teams'][1]['objectives']['riftHerald']['first'] == True:
                    match['FIRST_HERALD'] = ['RED']
                else:
                    match['FIRST_HERALD'] = ['NONE']
    
                match['BHERALD_KILLS'] = match_info['info']['teams'][0]['objectives']['riftHerald']['kills']
                match['RHERALD_KILLS'] = match_info['info']['teams'][1]['objectives']['riftHerald']['kills']

                # Assign who got First Tower and get the total tower kills for each team
                if match_info['info']['teams'][0]['objectives']['tower']['first'] == True:
                    match['FIRST_TOWER'] = ['BLUE']
                elif match_info['info']['teams'][1]['objectives']['tower']['first'] == True:
                    match['FIRST_TOWER'] = ['RED']
                else:
                    match['FIRST_TOWER'] = ['NONE']

                match['BTOWER_KILLS'] = match_info['info']['teams'][0]['objectives']['tower']['kills']
                match['RTOWER_KILLS'] = match_info['info']['teams'][1]['objectives']['tower']['kills']

                # Assign who got First Baron and get the total baron kills for each team
                if match_info['info']['teams'][0]['objectives']['baron']['first'] == True:
                    match['FIRST_BARON'] = ['BLUE']
                elif match_info['info']['teams'][1]['objectives']['baron']['first'] == True:
                    match['FIRST_BARON'] = ['RED']
                else:
                    match['FIRST_BARON'] = ['NONE']
    
                match['BBARON_KILLS'] = match_info['info']['teams'][0]['objectives']['baron']['kills']
                match['RBARON_KILLS'] = match_info['info']['teams'][1]['objectives']['baron']['kills']

                # Assign who got First Inhibitor and get the total inhibitor kills for each team
                if match_info['info']['teams'][0]['objectives']['inhibitor']['first'] == True:
                    match['FIRST_INHIB'] = ['BLUE']
                elif match_info['info']['teams'][1]['objectives']['inhibitor']['first'] == True:
                    match['FIRST_INHIB'] = ['RED']
                else:
                    match['FIRST_INHIB'] = ['NONE']
    
                match['BINHIB_KILLS'] = match_info['info']['teams'][0]['objectives']['inhibitor']['kills']
                match['RINHIB_KILLS'] = match_info['info']['teams'][1]['objectives']['inhibitor']['kills']

                # Determine which team won and assign it to the winner ('target') variable
                if match_info['info']['teams'][0]['win'] == True:
                    match['WINNER'] = ['BLUE']
                elif match_info['info']['teams'][1]['win'] == True:
                    match['WINNER'] = ['RED']
    
                matches_df = pd.concat([matches_df, match])

            except: 
                pass

            time.sleep(120)
        return matches_df

## EUW DATA

In [14]:
euw_players = [
    ['ULTRAMENTAL69', 'MEGAMENTAL69', 'Razørk Activoo', 'Yaspooky', 'Vinxen'],
    ['MrDon001', '89 fds 126', 'Ginrais', 'WD Inter', 'Fatihcan'],
    ['WAO RANK 1', 'SHO BEST COACH', 'la Quinqui', 'Fade 7', 'SekstiNi'],
    ['VeigarVu', 'KC Hantera', 'Crrazy', 'Batuuu', 'Enemy Team Riven'],
    ['Ducky x', 'heiii', 'I Will Perform', '2023WORLDS', 'Lonely Moonrise'],
    ['Theoloris', 'Lmzs', 'Eragon', 'AquaCloak', 'FTW OMEN'],
    ['GREATMENTAL69', 'elyoyaaaaaaaaaaa', 'Odoamne', 'im bad at math', 'Fulfilled Wish'],
    ['Sad Capybara', 'weakside sion', 'v9s Vibe', '15yo smurf', 'FTW OMEN'],
    ['Prominence1', 'Dimill 777', 'Thaygod', 'eu top1', 'TheDisconnect'],
    ['Erdotee', 'BSC Coldraa', 'Mr Tickle', 'FNC Upset03', 'VOICES TAKE OVER'],
]

In [15]:
euw_puuids = summoner_puuids(euw_players, 'euw1')

In [25]:
euw_match_ids = match_ids(euw_puuids, 'europe')

In [29]:
euw_match_date = match_data(euw_match_ids, 'europe')

In [34]:
euw_match_date.shape

(79, 50)